# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import numpy as np
import pandas as pd
import joblib
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import TruncatedSVD

nltk.download(['stopwords', 'wordnet', 'punkt'], quiet=True)

True

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM disaster_response', engine)
df.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message'].values
Y = df.drop(['message', 'genre'], axis=1).values

In [4]:
classes = [np.unique(Y[:, i]) for i in range(Y.shape[1])]
classes

[array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1])]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    '''
    Processes text to list of tokens

    INPUT:
    text

    OUTPUT:
    list - tokens generated from the text
    '''

    # normalize and tokenize text
    tokens = word_tokenize(text.lower())

    # lemmatize and remove stopwords
    tokens = [WordNetLemmatizer().lemmatize(word)
              for word in tokens if word not in stopwords.words('english')]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', TfidfVectorizer(tokenizer=tokenize)),
                     ('dim_red', TruncatedSVD(500)),
                     ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1,
                                                                          class_weight='balanced')))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25, random_state=42)

In [8]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7fc760403a60>)),
                ('dim_red', TruncatedSVD(n_components=500)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        n_jobs=-1)))])

In [9]:
Y_pred = pipeline.predict(X_test)
Y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
precision = []
recall = []
for i in range(Y_test.shape[1]):
    print(classification_report(Y_test[:, i], Y_pred[:, i], zero_division=0))
    precision.append(precision_score(
        Y_test[:, i], Y_pred[:, i], zero_division=0))
    recall.append(recall_score(Y_test[:, i], Y_pred[:, i], zero_division=0))

              precision    recall  f1-score   support

           0       0.73      0.13      0.23      1557
           1       0.78      0.98      0.87      4988

    accuracy                           0.78      6545
   macro avg       0.76      0.56      0.55      6545
weighted avg       0.77      0.78      0.72      6545

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5393
           1       0.85      0.27      0.41      1152

    accuracy                           0.86      6545
   macro avg       0.86      0.63      0.67      6545
weighted avg       0.86      0.86      0.83      6545

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

              preci

In [11]:
print('Accuracy: ', (Y_test == Y_pred).mean())
print('Precision: ', np.mean(precision))
print('Recall: ', np.mean(recall))

Accuracy:  0.9368432221373398
Precision:  0.4663812282150035
Recall:  0.11136190994345961


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline2 = Pipeline([('vect', TfidfVectorizer(tokenizer=tokenize)),
                      ('dim_red', TruncatedSVD(350)),
                      ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1,
                                                                           min_samples_leaf=75,
                                                                           class_weight='balanced')))])

parameters = {'dim_red__n_components': [150, 250, 350],
              'clf__estimator__min_samples_leaf': [75, 100, 150]}

cv = GridSearchCV(pipeline2, parameters)

In [15]:
cv.fit(X_train, Y_train)
cv.best_params_

{'clf__estimator__min_samples_leaf': 75, 'dim_red__n_components': 350}

In [16]:
Y_pred = cv.predict(X_test)
Y_pred

array([[0, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
precision = []
recall = []
for i in range(Y_test.shape[1]):
    print(classification_report(Y_test[:, i], Y_pred[:, i], zero_division=0))
    precision.append(precision_score(
        Y_test[:, i], Y_pred[:, i], zero_division=0))
    recall.append(recall_score(Y_test[:, i], Y_pred[:, i], zero_division=0))

print('Accuracy: ', (Y_test == Y_pred).mean())
print('Precision: ', np.mean(precision))
print('Recall: ', np.mean(recall))

              precision    recall  f1-score   support

           0       0.53      0.64      0.58      1557
           1       0.88      0.82      0.85      4988

    accuracy                           0.78      6545
   macro avg       0.70      0.73      0.72      6545
weighted avg       0.80      0.78      0.79      6545

              precision    recall  f1-score   support

           0       0.94      0.82      0.87      5393
           1       0.47      0.75      0.57      1152

    accuracy                           0.81      6545
   macro avg       0.70      0.78      0.72      6545
weighted avg       0.86      0.81      0.82      6545

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

              preci

### 8. Export your model as a pickle file

In [18]:
joblib.dump(cv.best_estimator_, 'classifier.pkl', compress=3)

['classifier.pkl']